In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [21]:
load_dotenv()

True

In [22]:
apikey = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=apikey
)
print(client)

In [24]:
chat_completion_stream = client.chat.completions.create(
    messages=[
        {"role":"user",
         "content":"Greet me.",
         },
    ],
    model="gpt-3.5-turbo",
    stream=True
)
for chunk in chat_completion_stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


Hello! How can I assist you today?